In [3]:
%pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 176.2 kB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 3.1 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 3.3 MB/s eta 0:00:003.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 6.8 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
keys = {}
with open('keys.json', 'r') as file:
    keys = json.load(file)

In [5]:
print(keys['taxNamePath'])
print(keys['taxNodePath'])

./taxdump/names.dmp
./taxdump/nodes.dmp


In [6]:
# set file path (up to your device)
file_path_names = keys['taxNamePath']
file_path_nodes = keys['taxNodePath']

In [7]:
# read data from path above
name_df = pd.read_table(file_path_names)
node_df = pd.read_table(file_path_nodes)


In [6]:
# rename column of nodes
relation_nodes = node_df[['1', '1.1', 'no rank']]
relation_nodes.rename(columns = {'1':'id'}, inplace = True)
relation_nodes.rename(columns = {'1.1':'parent_id'}, inplace = True)
relation_nodes.rename(columns = {'no rank':'rank'}, inplace = True)

# rename column of name
# use only row that sysnonym is scientific name for unique tax_id
name_df = name_df[name_df['synonym'] == 'scientific name']
name_df = name_df.reset_index()
name_df = name_df[['1', 'all']]
name_df.rename(columns = {'1':'id'}, inplace = True)
name_df.rename(columns = {'all':'name'}, inplace = True)

/var/folders/02/py43_20s4bvcf7c6xm5lbkd00000gn/T/ipykernel_37768/2777069957.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relation_nodes.rename(columns = {'1':'id'}, inplace = True)
/var/folders/02/py43_20s4bvcf7c6xm5lbkd00000gn/T/ipykernel_37768/2777069957.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relation_nodes.rename(columns = {'1.1':'parent_id'}, inplace = True)
/var/folders/02/py43_20s4bvcf7c6xm5lbkd00000gn/T/ipykernel_37768/2777069957.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

In [7]:
# convert rank name to rank number
# map_rankName_value = {'species': 7, 'genus': 6, 'family': 5, 'order': 4, 'class': 3, 'phylum': 2, 'kingdom': 1, 'clade': 0, 'superkingdom': -1}
# relation_nodes['rank'] = relation_nodes['rank'].replace(map_rankName_value)

In [8]:
relation_nodes

,id,parent_id,rank
0,2,131567,superkingdom
1,6,335928,genus
2,7,6,species
3,9,32199,species
4,10,1706371,genus
...,...,...,...
2520227,3070616,3059992,no rank
2520228,3070820,3060827,no rank
2520229,3070859,4783,species
2520230,3070949,82115,genus


In [9]:
name_df

,id,name
0,1,root
1,2,Bacteria
2,6,Azorhizobium
3,7,Azorhizobium caulinodans
4,9,Buchnera aphidicola
...,...,...
2520123,3070616,Gordonia phage EricDab
2520124,3070820,Staphylococcus virus IME1354_01
2520125,3070859,Phytophthora novae-guineae
2520126,3070949,Flavimaribacter


In [10]:
# create df for map rank name and rank no.
map_rankName_value = {'species': 7, 'genus': 6, 'family': 5, 'order': 4, 'class': 3, 'phylum': 2, 'kingdom': 1, 'clade': 0, 'superkingdom': -1}
map_rankName_value = sorted(list(map_rankName_value.items()), key=lambda x:x[1])
df_map_rank_value = pd.DataFrame(map_rankName_value, columns=['name', 'rank no'])

In [11]:
df_map_rank_value

,name,rank no
0,superkingdom,-1
1,clade,0
2,kingdom,1
3,phylum,2
4,class,3
5,order,4
6,family,5
7,genus,6
8,species,7


In [14]:
list(name_df.columns.values)

['id', 'name']

In [66]:
def createManyFakeData(data):
    prev = 0
    columnValue = list(data.columns.values)
    newData = []
    for d in data.values:
        dData = {k: int(v) if k == 'parent_id' or k == 'id' else v for k, v in zip(columnValue, d)}
        for i in range(prev, int(d[0])):
            fakeData = {k: 0 for k in columnValue}
            fakeData['id'] = i
            newData.append(fakeData)
        newData.append(dData)
        prev = int(d[0])+1
    return pd.DataFrame(newData)


In [67]:
newNames_df = createManyFakeData(name_df)
newRelations_df = createManyFakeData(relation_nodes)


In [71]:
newNames_df[newNames_df['name'] == 0]

,id,name
0,0,0
3,3,0
4,4,0
5,5,0
8,8,0
...,...,...
3070948,3070948,0
3070950,3070950,0
3070951,3070951,0
3070952,3070952,0


In [69]:
newRelations_df

,id,parent_id,rank
0,0,0,0
1,1,0,0
2,2,131567,superkingdom
3,3,0,0
4,4,0,0
...,...,...,...
3070950,3070950,0,0
3070951,3070951,0,0
3070952,3070952,0,0
3070953,3070953,0,0


In [8]:
destScName = keys['destScientificNameFile']
destRelation = keys['destRelationFile']
print(destScName)
print(destRelation)

./data/scientific_name.csv
./data/relation.csv


In [48]:
newNames_df.to_csv(destScName, index=False)
newRelations_df.to_csv(destRelation, index=False)

In [49]:
relation_nodes['rank'].unique()

array(['superkingdom', 'genus', 'species', 'order', 'family',
       'subspecies', 'no rank', 'subfamily', 'strain', 'serogroup',
       'biotype', 'tribe', 'phylum', 'class', 'species group', 'forma',
       'clade', 'suborder', 'subclass', 'varietas', 'kingdom',
       'subphylum', 'forma specialis', 'isolate', 'superfamily',
       'infraorder', 'infraclass', 'superorder', 'subgenus', 'superclass',
       'parvorder', 'serotype', 'species subgroup', 'subcohort', 'cohort',
       'genotype', 'subtribe', 'section', 'series', 'morph', 'subkingdom',
       'superphylum', 'subsection', 'pathogroup'], dtype=object)